# Fantasy Hockey Data Wrangling

In this notebook we will wrangle the data used for the Fantasy Hockey draft. This will involve a few steps
1. Gather various tables of players, teams, and salaries
2. Join the tables
3. Calculate each player's score-per-game with custom scoring metrics for our fantasy hockey league 

Note that many of the helper scripts have been abstracted away to `scripts/hockey_bots.py`


## Collecting Data

Below we import libraries, as well as import required data into the notebook. 

In [62]:
import pandas as pd
import numpy as np
import sys
import importlib
sys.path.insert(1, '../')
import scripts.hockey_bots as hockey
importlib.reload(hockey)

# players table (stats)
df_p = pd.read_csv("../data/game_skater_stats.csv")
# game data dable
df_g = pd.read_csv("../data/game.csv")
# goalies table 
df_go = pd.read_csv("../data/game_goalie_stats.csv")
# player/goalie table (name, team, etc)
df_player = pd.read_csv("../data/player_info.csv")
# shifts table 
shifts = pd.read_csv("../data/game_shifts.csv")


### Goalies

As goalies are awarded points for starting a game, we need to filter our shifts table to goalies and first period to see when they start.

In [63]:
# Figuring out if a goalie started a game or not (starting the game is worth points)
goal_shifts = shifts[shifts.player_id.isin(df_go.player_id)]
goal_shifts = goal_shifts[goal_shifts.period==1]


### Filtering to the 2018-2019 Season

To simplify our analysis ,we will only foucs on the 2018-2019 hockey season. This is done below

In [40]:
import datetime

# Finding data for just this season
datetime.datetime.strptime
df = pd.read_csv('../data/game_teams_stats.csv')
df =  pd.merge(df, df_g[['game_id', 'date_time', 'type']])
df['date_time'] = pd.to_datetime(df['date_time'])

df = df[(df['date_time'] > '2018-10-3') &
        (df['date_time'] < '2019-04-10') & 
        (df['type'] == 'R')]
df = df.sort_values(by=['team_id', 'date_time'])
df['game_num'] = df.groupby('team_id').cumcount()

In [41]:
df_games = df[['game_id','team_id', 'won', 'game_num']]


### Salary Information

As a constraint, we may want to include a maximum salary for our fantasy hockey team. To do this we gather hockey salaries below and save them to a data frame

In [42]:
salaries = pd.read_html('https://www.hockey-reference.com/friv/current_nhl_salaries.cgi')[0]
salaries['firstName'], salaries['lastName'] = salaries['Player'].str.split(' ', 1).str


### Merging Tables
Below we merge the player tables, thir salaries and other information into a single table.

In [64]:
#players 
df_p_2018 = hockey.player_merge(df_p, df_g, df_player, salaries)
df_p_2018 =df_p_2018[(df_p_2018['date_time'] > '2018-10-3') &
           (df_p_2018['date_time'] < '2019-04-10') & 
           (df_p_2018['type'] == 'R')]

# goalies

df_g_2018 = hockey.player_merge(df_go, df_g, df_player, salaries)
df_g_2018 =df_g_2018[(df_g_2018['date_time'] > '2018-10-3') &
           (df_g_2018['date_time'] < '2019-04-10') & 
           (df_g_2018['type'] == 'R')]

In [44]:
df_p_2018['points'] = df_p_2018.copy().apply(hockey.player_points, axis=1)
df_g_2018['points'] = df_g_2018.copy().apply(hockey.goalie_points, args=[goal_shifts], axis=1)

In [45]:
import matplotlib.pyplot as plt
df_p_2018=df_p_2018.sort_values(by='date_time').reset_index(drop=True)
df_g_2018=df_g_2018.sort_values(by='date_time').reset_index(drop=True)
df_score = df_p_2018[['game_id', 'team_id', 'player_id','Salary','firstName', 'lastName', 'primaryPosition', 'points']]
df_scoreg = df_g_2018[['game_id', 'team_id', 'player_id','Salary','firstName', 'lastName', 'primaryPosition', 'points']]
df_score=df_score.append(df_scoreg, ignore_index=True)

In [47]:
a = pd.merge(df_score, df_games, on = 'game_id', how='left')


In [65]:
a=a.sort_values(by=['player_id', 'game_num'], ascending=False).drop_duplicates(subset=['player_id','game_num'])
a.head()

,game_id,team_id_x,player_id,Salary,firstName,lastName,primaryPosition,points,team_id_y,won,game_num
70862,2018021270,26,8481481,925000,Blake,Lizotte,C,0.03,26,True,81
64804,2018021165,53,8480950,874125,Ilya,Lyubushkin,D,0.24,2,True,75
60566,2018021090,53,8480950,874125,Ilya,Lyubushkin,D,1.56,24,False,71
60567,2018021090,53,8480950,874125,Ilya,Lyubushkin,D,1.56,53,True,70
60075,2018021076,53,8480950,874125,Ilya,Lyubushkin,D,-0.91,53,True,69


## Final Data Merge

In this case we need to add empty (zero filled) rows for players who did not play in a particular game. This is important as we need to have an equal amount of games played for each player for our portfolio optimization later. Note that zero filling is not _necessarily_ the best thing to do. One could also fill by the mean/median/some other metric. In this case zero filling was chosen as if we're looking to pick a player for fantasy hockey - if a player isn't playing many games, they're not going to help us win.

In [49]:
games = list(a.game_num.unique())
test = a.copy()
for player in a['player_id'].unique():
    games_played =  list(a[a['player_id'] == player]['game_num'])
    fill_games = list(set(games) - set(games_played))
    for game in fill_games:
        pos = a[a['player_id'] == player]['primaryPosition'].to_list()[0]
        salary = a[a['player_id'] == player]['Salary'].to_list()[0]
        first =  a[a['player_id'] == player]['firstName'].to_list()[0]
        last =   a[a['player_id'] == player]['lastName'].to_list()[0]
        to_append = pd.DataFrame([[np.nan, 
                                   np.nan, 
                                   player, 
                                   salary, 
                                   first,
                                   last,
                                   pos, 
                                   0, 
                                   np.nan,
                                   np.nan,
                                   game]], 
                                 columns = list(a))
        test = test.append(to_append, ignore_index=True)
    
        


test.head()

,game_id,team_id_x,player_id,Salary,firstName,lastName,primaryPosition,points,team_id_y,won,game_num
0,2.018021e+09,26.0,8481481,925000,Blake,Lizotte,C,0.03,26.0,1.0,81
1,2.018021e+09,53.0,8480950,874125,Ilya,Lyubushkin,D,0.24,2.0,1.0,75
2,2.018021e+09,53.0,8480950,874125,Ilya,Lyubushkin,D,1.56,24.0,0.0,71
3,2.018021e+09,53.0,8480950,874125,Ilya,Lyubushkin,D,1.56,53.0,1.0,70
4,2.018021e+09,53.0,8480950,874125,Ilya,Lyubushkin,D,-0.91,53.0,1.0,69
...,...,...,...,...,...,...,...,...,...,...,...
50261,NaN,NaN,8465009,2000000,Zdeno,Chara,D,0.00,NaN,NaN,27
50262,NaN,NaN,8465009,2000000,Zdeno,Chara,D,0.00,NaN,NaN,28
50263,NaN,NaN,8465009,2000000,Zdeno,Chara,D,0.00,NaN,NaN,29
50264,NaN,NaN,8465009,2000000,Zdeno,Chara,D,0.00,NaN,NaN,30


### Dropping Players
Here we're ignoring any player that did not play more than 10 games in the previous season.

In [50]:
grouped = test.groupby('player_id').count()
players=list(grouped[grouped['won'] > 10].index)
test2=test[test['player_id'].isin(players)].reset_index(drop=True)

In [52]:
# Zero fillign 
test = test.fillna(0)
test = test.sort_values(by=['player_id', 'game_num'])
len(test.player_id.unique())

test2 = test2.fillna(0)
test = test2.sort_values(by=['player_id', 'game_num'])
len(test2.player_id.unique())


588

In [54]:
test.to_csv("../data/points_w_salary_and_position2.csv")

In [55]:
test.to_csv("../data/scores2.csv", index=False)

In [57]:
scores = test2[['player_id', 'points',]].groupby('player_id').agg(lambda x: list(x)).reset_index()
#pd.DataFrame(np.array(scores.points.tolist()), columns=test.player_id.tolist())
all_points = pd.DataFrame(np.transpose(scores.points.tolist()), columns = scores.player_id)
idx = list(all_points.mean().sort_values(ascending=False).index)


In [60]:
test2.to_csv("../data/player_data_for_optim3.csv")

In [61]:
a.to_csv("../data/textaa.csv")